In [1]:
import tkinter as tk
import tkinter.ttk as ttk

In [25]:
class EventDispatcher():
    def __init__(self):
        self.handlers = {}
        
    def create_event(self, ev):
        self.handlers[ev] = self.handlers.get(ev, [])
        
    def register(self, ev, handler):
        if ev not in self.handlers:
            self.create_event(ev)
        self.handlers[ev].append(handler)
        
    def unregister(self, ev, handler):
        self.handlers[ev].remove(handler)
        
    def fire(self, ev, *args):
        for h in self.handlers.get(ev, ()):
            h(ev, *args)
            
    def unregister_all(self, ev):
        self.handlers[ev] = []
        
    def create_callback(self, ev, *args):
        def cb():
            self.fire(ev, *args)
        return cb

In [48]:
Events = EventDispatcher()  # global

class CommandButton(ttk.Button):
    def __init__(self, master, text, cb, row=1, col=1):
        super().__init__(master, text=text, command=cb)
        self.row = 1
        self.col = 1
        
    def grid_loc(self, row=None, col=None):
        if row is not None: self.row = row
        if col is not None: self.col = col
            
    def grid(self, row=None, col=None, **ops):
        if row is None:
            row = self.row
        if col is None:
            col = self.col
        ops['row'] = row
        ops['column'] = col
        super().grid(**ops)
        
    def disable(self):
        self.config(state='disabled')
    
    def enable(self):
        self.config(state='enabled')
        
class SimpleFrame(ttk.Frame):
    def grid(self, row=None, col=None, **kw):
        if row is not None:
            kw['row'] = row
        if col is not None:
            kw['column'] = col
        super().grid(**kw)

class App():
    def __init__(self):
        self.tk = tk.Tk()
        self.mf = SimpleFrame(self.tk)
        self.next = CommandButton(self.tk, "Next", self.next_menu)
        self.back = CommandButton(self.tk, "Back", self.prev_menu)
        self.back.disable()
        self.menus = []
        self.current_menu = 0
    
    def add_menu(self, m):
        self.menus.append(m)
        
    def next_menu(self):
        
        c = self.current_menu
        if c >= len(self.menus) - 1:
            print("WARNING: next_menu called from last menu!")
        self.menus[c].grid_forget()
        self.menus[c+1].grid()
        self.current_menu += 1
        if self.current_menu >= len(self.menus) - 1:
            self.next.disable()
        else:
            self.next.enable()
        self.back.enable()
            
    def prev_menu(self):
        c = self.current_menu
        if c <= 0:
            print("WARNING: prev_menu called from first menu!")
        self.menus[c].grid_forget()
        self.menus[c-1].grid()
        self.current_menu -= 1
        if self.current_menu == 0:
            self.back.disable()
        else:
            self.back.enable()
        self.next.enable()
            
            
    def Menu(self, cls, *args, **kw):
        self.add_menu(cls(self.mf, *args, **kw))
        
    def run(self):
        self.grid()
        self.tk.mainloop()
    
    def grid(self):
        self.mf.grid(1, 1)
        self.back.grid(2, 1)
        self.next.grid(2, 2)
        self.menus[self.current_menu].grid()
        
class Menu():
    def __init__(self, master):
        self.frame = tk.Frame(master)
        
    def grid(self, **ops):
        self.frame.grid(**ops)
        
    def grid_forget(self):
        self.frame.grid_forget()
        

In [49]:
class TestMenu(Menu):
    def __init__(self, master, text):
        super().__init__(master)
        self.text = ttk.Label(self.frame, text=text)
        self.text.grid()
Events = EventDispatcher()  # global
app = App()
app.Menu(TestMenu, "Hello World")
app.Menu(TestMenu, "Foo World!")
app.Menu(TestMenu, "Bye World!")
app.run()

1 3
2 3
1 3
2 3
1 3
2 3
